In [3]:
import pandas as pd
import pandahouse as ph

Создаем словарь с нужными подключениями

In [178]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Посмотрим на данные.

In [179]:
q = '''
    SELECT
        st_id,
        timest::DATETIME AS timest,
        correct,
        subject        
    FROM default.peas
    LIMIT 5
    '''

peas = ph.read_clickhouse(query=q, connection=connection_default)
peas

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


In [180]:
q = '''
    SELECT
        st_id,
        test_grp       
    FROM default.studs
    LIMIT 5
    '''

studs = ph.read_clickhouse(query=q, connection=connection_default)
studs

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot


In [181]:
q = '''
    SELECT
        st_id,
        sale_time::DATETIME AS sale_time, 
        money,
        subject
    FROM default.final_project_check
    LIMIT 5
    '''

final_project_check = ph.read_clickhouse(query=q, connection=connection_default)
final_project_check

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization
3,104885,2021-10-30 22:49:33,75000,Statistics
4,106464,2021-10-31 13:17:13,85000,Math


Необходимо понять временной промежуток. Для этого посмотрим минимальную и максимальную даты.

In [182]:
q = '''
    SELECT
        MIN(timest::DATE) AS min_date,
        MAX(timest::DATE) AS max_date
    FROM default.peas
    '''

dates_1 = ph.read_clickhouse(query=q, connection=connection_default)
dates_1

,min_date,max_date
0,2021-10-30,2021-10-31


In [183]:
q = '''
    SELECT
        MIN(sale_time::DATE) AS min_date,
        MAX(sale_time::DATE) AS max_date
    FROM default.final_project_check
    '''

dates_2 = ph.read_clickhouse(query=q, connection=connection_default)
dates_2

,min_date,max_date
0,2021-10-30,2021-10-31


Данные лежат в пределах одного месяца.

Найдем количество очень усердных студентов.

В данном случае нам важно общее количество правильно решенных горошин у каждого студента вне зависимости от предмета. Так как правильность решения "горошины" обозначена 0 и 1, то на достаточно суммировать correct для каждого студента, а затем посчитать кол-во студентов, у которых сумма >=20. 

In [184]:
q = '''
    WITH correct_peas_count AS
    (SELECT
        st_id,
        SUM(correct) AS correct_p
    FROM default.peas    
    GROUP BY st_id
    )
    
    SELECT
        COUNT(st_id) AS diligent_st_count
    FROM correct_peas_count   
    WHERE correct_p >= 20
    '''
diligent_students = ph.read_clickhouse(query=q, connection=connection_default)
diligent_students

,diligent_st_count
0,136


Всего 136 усердных учеников.

Напишем запрос для нахождения всех необходимых метрик

In [185]:
q = '''
    WITH all_data AS (
    
    SELECT
        s.st_id AS st_id,
        test_grp,
        money,
        subject,
        correct_p,
        correct_math_p
    FROM studs s
    LEFT JOIN final_project_check f ON s.st_id = f.st_id
    LEFT JOIN (
                SELECT 
                    st_id, 
                    SUM(correct) AS correct_p,
                    SUM(correct) FILTER(WHERE subject='Math') as correct_math_p
                FROM peas
                GROUP BY st_id
                ) AS p 
    ON s.st_id = p.st_id
)

SELECT
    test_grp,
    ROUND(SUM(money) / COUNT(DISTINCT st_id), 2) AS ARPU, -- средняя выручка на одного пользователя
    ROUND(SUM(money) FILTER(WHERE correct_p > 10) / COUNT(DISTINCT st_id) FILTER(WHERE correct_p > 10), 2) AS ARPAU, -- средняя выручка на одного активного пользователя
    ROUND(COUNT(DISTINCT st_id) FILTER(WHERE money > 0) / COUNT(DISTINCT st_id) * 100, 2) AS CR, -- отношение числа тех, кто совершил покупку, и тех, кто посетил сайт
    ROUND(COUNT(DISTINCT st_id) FILTER(WHERE money > 0 AND correct_p > 10) / COUNT(DISTINCT st_id) FILTER(WHERE correct_p > 10) * 100, 2) AS CR_active,
    ROUND(COUNT(DISTINCT st_id) FILTER(WHERE money > 0 AND subject = 'Math' AND correct_math_p >= 2) / COUNT(DISTINCT st_id) FILTER(WHERE correct_math_p >= 2) * 100, 2) AS CR_math
FROM all_data
GROUP BY test_grp
    '''
funnel = ph.read_clickhouse(query=q, connection=connection_default)
funnel

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52


В группе pilot, где использовалась модель trial, все метрики увеличились. 